In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from IPython.core.display import HTML

In [0]:
# Read recipe inputs
House_and_SaleInfo = dataiku.Dataset("House_and_SaleInfo")
House_and_SaleInfo_df = House_and_SaleInfo.get_dataframe()

In [0]:
#get the list of columns you want to create freq reports for
project_handle = dataiku.api_client().get_project(dataiku.default_project_key())
list_of_columns = project_handle.get_variables()['local']['list_of_columns']
print(list_of_columns)

In [0]:
#function to create a single freq report for a column
def single_freq(df, var1):
    freq_counts = df[var1].value_counts(dropna=False)
    df = pd.concat([freq_counts], axis=1, keys=['Frequency'])
    df["Cumulative Frequency"] = df["Frequency"].cumsum()

    style_caption = dict(selector="caption",
                        props=[("text-align", "right"),
                               ("font-size", "120%"),
                               ("color", 'blue')])
    style_td = dict(selector="td", props=[("border", "1px solid grey"),
                                          ("cellpadding","0")])
    style_th = dict(selector="th", props=[("border", "1px solid grey"),
                                          ("cellpadding","0")])


#                                           ('border-color', 'black'),
#                                           ('border-style', 'solid')])
#                                           ('border-width', '1px')])

    styles = [style_caption, style_td, style_th]

    df = df.style.set_table_attributes("style='display:inline'").set_caption('Frequency Table for '+var1).set_table_styles(styles)
    df_html = df.render()
    return df_html
    #return df

In [0]:
#Test the function
single_freq(House_and_SaleInfo_df,"Land Slope")

In [0]:
#loop through every column in the list, create the html string for the related freq report,
#append to the string to create the full report
full_report = ''
for column in list_of_columns:
    individual_freqs = single_freq(House_and_SaleInfo_df, column)
    full_report += individual_freqs + '<br><br>'

In [0]:
# print(full_report)
#show what it looks like
HTML(full_report)

In [0]:
#create the HTML file and add the HTML file to managed folder
HTML_report = dataiku.Folder("0HZULy8W")

Home_Sale_Freq_Report = str.encode(full_report)

with HTML_report.get_writer("Home_Sale_Freq_Report.html") as w:
    w.write(Home_Sale_Freq_Report)